In [1]:
import spacy 
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
from spacy.vectors import Vectors

In [3]:
text=spacy.load("en_core_web_sm")
file=pd.read_csv('./netflix_titles.csv')
file.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [4]:
file.shape

(6234, 12)

In [12]:
vectorize1=[text(x).vector for x in file[file['type']=='Movie']['description'].values]

In [11]:
vectorize1=np.stack(vectorize1,axis=0)
vectorize1.shape

(4265, 96)

In [13]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=10)
sentence_svd_vector=svd.fit_transform(vectorize1)
sentence_svd_vector

array([[ 6.26121   , -1.8026189 , -0.4910885 , ..., -0.7170616 ,
        -0.0196943 ,  0.16006358],
       [ 5.848837  ,  0.69792295,  0.3127798 , ..., -0.29406717,
         0.58530015, -0.8719191 ],
       [ 5.614216  ,  1.1314791 , -0.3500703 , ...,  0.3495914 ,
        -0.28252536, -0.49713808],
       ...,
       [ 6.901638  ,  1.04107   , -1.8079075 , ...,  0.9714138 ,
        -0.41830885,  0.5008518 ],
       [ 7.7624044 ,  1.640588  , -1.0129762 , ..., -0.6171848 ,
         0.51592577, -0.0714866 ],
       [ 5.718037  ,  3.0807092 ,  3.6097176 , ...,  0.68482804,
         0.5241602 , -0.19361375]], dtype=float32)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim=cosine_similarity(sentence_svd_vector,sentence_svd_vector)
pd.DataFrame(cos_sim)[10].sort_values(ascending=False)

10      1.000000
2391    0.985628
12      0.977654
3920    0.972210
3306    0.969413
          ...   
3975    0.455881
186     0.455118
4059    0.431995
1841    0.419370
2255    0.366754
Name: 10, Length: 4265, dtype: float32

In [25]:
print(file[file['type']=='Movie']['description'][10],'\n')
print(file[file['type']=='Movie']['description'][2391],'\n')

Comedian and celebrity impersonator Joaquín Reyes decides to be his zesty self for a night of stories about buses, bathroom habits, royalty and more. 

The boys head to Ireland after winning a contest to see Rush but are arrested by immigration and must perform a community service puppet show. 



In [19]:
vectorize2=np.stack([text("Soldiers on the island realize that they are in an experiment with robots during training.").vector])
vectorize2=np.append(vectorize1,vectorize2,axis=0)
vectorize2.shape

(4266, 96)

In [20]:
sentence_svd_vectorize=svd.fit_transform(vectorize2)
CosSim=cosine_similarity(sentence_svd_vectorize,sentence_svd_vectorize)
pd.DataFrame(CosSim)[4265].sort_values(ascending=False)

4265    1.000000
4221    0.986794
3066    0.985523
161     0.984413
330     0.980666
          ...   
2098    0.511878
4190    0.509562
136     0.503548
2763    0.496545
1339    0.484788
Name: 4265, Length: 4266, dtype: float32

In [21]:
print(file[file['type']=='Movie']['description'][4221])

Abandoned by her new husband and father of her son, a mountain woman journeys to Calcutta to find him, constantly tested by social evils along the way.
